# Web Tree 구축

In [1]:
import os
import json
import time

from collections import deque
from urllib.parse import urljoin, urlparse

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
def set_webdriver(root_url):
    options = Options()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    options.add_argument(f"user-agent={user_agent}")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(root_url)
    time.sleep(0.5)

    return driver

def initialize_crawler(start_url):
    """크롤러 초기화: 방문한 URL, 트리 구조 및 BFS 큐 설정"""
    visited = set([start_url])
    tree = {}
    queue = deque([(start_url, None)])
    return visited, tree, queue

In [ ]:
def get_page_content(driver, url):
    """URL을 로드하고 페이지 내용을 가져옴"""
    try:
        driver.get(url)
        time.sleep(0.5)
        return driver.find_element(By.TAG_NAME, "body").text
    except Exception as e:
        print(f"페이지 로드 오류: {e} (URL: {url})")
        return ""

def process_links(driver, current_url, start_url, visited):
    """현재 페이지의 모든 링크를 찾고 BFS 큐에 추가"""
    links_to_visit = []
    try:
        links = driver.find_elements(By.TAG_NAME, "a")
        for link in links:
            href = link.get_attribute("href")
            if href:
                absolute_url = urljoin(current_url, href)
                parsed_url = urlparse(absolute_url)
                clean_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
                if parsed_url.netloc == urlparse(start_url).netloc and clean_url not in visited:
                    visited.add(clean_url)
                    links_to_visit.append(clean_url)
    except Exception as e:
        print(f"링크 처리 오류: {e} (URL: {current_url})")
    return links_to_visit

In [ ]:
def save_results(tree, filename="crawl_result.json"):
    """크롤링 결과를 JSON 파일로 저장"""
    with open(filename, "w", encoding="utf-8") as json_file:
        json.dump(tree, json_file, ensure_ascii=False, indent=4)
    print(f"크롤링 결과가 {filename} 파일로 저장되었습니다.")

In [ ]:
def crawl_website(start_url):
    """웹사이트를 BFS 방식으로 크롤링"""
    driver = set_webdriver(start_url)
    visited, tree, queue = initialize_crawler(start_url)
    
    while queue:
        current_url, parent = queue.popleft()
        page_text = get_page_content(driver, current_url)
        
        node = {"url": current_url, "text": page_text, "children": []}
        tree[current_url] = node
        
        if parent is not None:
            tree[parent]["children"].append(node)
        
        new_links = process_links(driver, current_url, start_url, visited)
        queue.extend((link, current_url) for link in new_links)
    
    driver.quit()
    return tree

In [ ]:
root_url = "https://audit.hyundai.com/"
crawl_result = crawl_website(root_url)
save_results(crawl_result, "../data/현대자동차그룹.json")

# RAG

## Document

In [1]:
import os
import json
import glob

from uuid import uuid4
from langchain.docstore.document import Document

In [2]:
json_files = sorted(glob.glob("../data/*_page_contents.json"))
print(len(json_files))
print(json_files)

3
['../data/sap_korea_page_contents.json', '../data/남양유업_page_contents.json', '../data/현대자동차그룹_page_contents.json']


In [3]:
def load_documents(file_list):
    total_docs = []
    for json_path in file_list:
        company_name = json_path.split('/')[-1].split('_')[0]
        with open(json_path, "r", encoding="utf-8") as f:
            page_contents = json.load(f)
            print(f"{company_name} - {len(page_contents)}")

        documents = [
            Document(
                page_content=text,
                metadata={"company_name" : company_name, "url": url},
                id=uuid4()
            )
            for url, text in page_contents.items()
        ]
        total_docs.extend(documents)

    return total_docs

In [4]:
documents = load_documents(json_files)
print(len(documents))

sap - 151
남양유업 - 151
현대자동차그룹 - 22
324


## Vector DB

### Elastic Search

### docker build
```bash
# docker pull docker.elastic.co/elasticsearch/elasticsearch-wolfi:8.17.1

docker network create elastic
docker pull docker.elastic.co/elasticsearch/elasticsearch:8.17.1

docker run --name es01 --net elastic \
-p 9200:9200 -it -m 6GB \
docker.elastic.co/elasticsearch/elasticsearch:8.17.1
```

``` bash
# ERROR: Elasticsearch died while starting up, with exit code 78 발생시
echo "vm.max_map_count=262144" | sudo tee -a /etc/sysctl.conf
sudo sysctl -p

docker run --name es01 --net elastic \
-p 9200:9200 -it -m 6GB \
docker.elastic.co/elasticsearch/elasticsearch:8.17.1
```

```bash
pip install -U langchain_elasticsearch
```

Reference
- [https://wikidocs.net/234016](https://wikidocs.net/234016)

In [5]:
import os
import faiss

from datetime import datetime

from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

from langchain_openai import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv("../keys.env")
OPENAI_API_KEY = os.getenv("GRAVY_LAB_OPENAI")

In [6]:
def load_embed_model(provider, model_name, api_key=None, model_kwargs=None, encode_kwargs=None):
    if provider == "openai":
        model = OpenAIEmbeddings(model=model_name, api_key=api_key)
    elif provider == "huggingface":
        model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

    return model

In [7]:
## faiss
def create_vector_db(index_type, embed_model):
    sample_query = "Hello_World!"

    if index_type == "L2": ## L2 Distance(Euclidean Distance)
        index = faiss.IndexFlatL2(len(embed_model.embed_query(sample_query)))

    elif index_type == "IP": ## Inner Product
        index = faiss.IndexFlatIP(len(embed_model.embed_query(sample_query)))

    elif index_type == "HNSW":## ANN -> Hierarchical Navigable Small World
        index = faiss.IndexHNSWFlat(len(embed_model.embed_query(sample_query)))

    vector_db = FAISS(
        embedding_function=embed_model,
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
    )

    return vector_db

In [8]:
def save_vector_db(vector_db, save_path):
    vector_db.save_local(save_path)

def load_vector_db(index_file_path, embed_model):
    vector_db = FAISS.load_local(index_file_path, embeddings=embed_model, allow_dangerous_deserialization=True)

    return vector_db

In [9]:
now = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
save_path = f"../indexes/{now}"
os.makedirs(save_path, exist_ok=True)

In [10]:
provider = "huggingface"
model_name = "intfloat/multilingual-e5-large-instruct"
model_kwargs = {"device" : "cuda"}
encode_kwargs = {"normalize_embeddings": True}
index_type = "IP" ## "L2", "IP", "HNSW"

embed_model = load_embed_model(provider,
                               model_name,
                               api_key=OPENAI_API_KEY,
                               model_kwargs=model_kwargs, 
                               encode_kwargs=encode_kwargs)

In [ ]:
## faiss

vector_db = create_vector_db(index_type, embed_model)
vector_db.add_documents(documents)
vector_db.save_local(save_path)

In [28]:
from elasticsearch import Elasticsearch
from langchain_elasticsearch import ElasticsearchStore

# SSL 인증서 검증 비활성화
es_client = Elasticsearch(
    ["https://es01:9200"],
    basic_auth=("elastic", "CLlD0d++O7Dvcy60VSAb"),
    verify_certs=False
)

vector_store = ElasticsearchStore(
    es_connection=es_client,
    index_name="langchain_index",
    embedding=embed_model,
)

/usr/local/lib/python3.10/dist-packages/elasticsearch/_sync/client/__init__.py:403: SecurityWarning: Connecting to 'https://es01:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [29]:
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


['48a2d385-d944-443b-b20f-ba304f33e6e4',
 '7bec982e-eeb4-471c-bdd3-f334316bbd6c',
 'b1d07924-f6ec-4fc1-9cc0-256019629481',
 'a48e3f36-00f2-4273-9e51-7108c6d2d352',
 '90ab4066-2625-4efc-9e32-bb5fac199238',
 '35b1fb59-e7fd-4297-ab1b-a066b54d4ebb',
 'ed8c2f08-e3c5-48b2-9d25-b5282ab60c23',
 '34961fde-79dd-4bb2-a91f-31ad2c5d4258',
 '793e0de0-0cfb-4e8a-b105-2e58c7b0852c',
 '727ec4b5-a8bb-4380-8bcc-6051b85833d7',
 '30fb8e40-cb27-4363-a29e-fb31656e2474',
 'e637af67-a7a7-4a2b-92da-60e55501268a',
 'd3cb009a-ab25-409a-ba1d-aba083fb3ba5',
 '8a3a3043-c2f3-4e0d-8fe7-ec4e9921dda2',
 'dce365ba-ad84-470a-a94b-349c2bd4c906',
 '2c85c664-a6f0-4fb4-919b-68ad2a7bcf29',
 '467baa93-42e7-4ea3-8219-c11e9b768df6',
 'b25a1ee1-2c59-44b8-813d-80c03de7c66a',
 '6d54ec80-ec12-43c7-a7fa-2dfeb9688873',
 '8965bb6b-7815-4d27-8415-db551a42a307',
 'e4115917-f3f8-4dbb-845b-119b20f99c51',
 '54d6d241-9d26-461e-a320-f537b06efb98',
 'e9d64d9a-4882-49d8-a3de-8a03af58c49a',
 '4ae4939c-1d28-40a5-bbf7-c9866d43fe62',
 '9904fde9-58aa-

In [30]:
indices = es_client.cat.indices(format="json")
print(indices)

[{'health': 'yellow', 'status': 'open', 'index': 'langchain_index', 'uuid': 'Am45VTcWQnijS-ubpvxSNw', 'pri': '1', 'rep': '1', 'docs.count': '324', 'docs.deleted': '0', 'store.size': '7.9mb', 'pri.store.size': '7.9mb', 'dataset.size': '7.9mb'}]


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [32]:
result = es_client.search(index="langchain_index", body={"query" : {"match_all" : {}}})
print(result)

{'took': 37, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 324, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'langchain_index', '_id': '48a2d385-d944-443b-b20f-ba304f33e6e4', '_score': 1.0, '_ignored': ['text.keyword'], '_source': {'text': '본 사이트에서는 사이트 운영, 분석, 사용자 경험 향상 또는 광고 등의 목적으로 쿠키 정책에 명시된 것처럼 쿠키 및 관련 기술을 사용하고 있습니다. 이러한 기술을 사용하는 데 동의하거나 기본 설정을 직접 조정할 수 있습니다.\n모두 허용\n모두 거부\n설정 관리\n개인정보 보호정책\n|\n쿠키 정책\n제품\n지원\n학습\n커뮤니티\n파트너\n정보\n전화:\n대한민국\n080-219-2400\n지역별 국가 번호 전체 목록 보기\n채팅 이용 불가\n채팅이 가능하도록 기능 쿠키를 허용하세요.\nSAP에 문의\n의견, 질문 또는 피드백을 보내주세요.\n모든 것을 연결하여\n무엇이든 이루세요\n60분 동안 진행되는 가상 이벤트에 참여해 SAP가 AI, 데이터, 애플리케이션을 결합해 귀사의 잠재력을 극대화하고 한계를 뛰어넘게 하는 방법을 알아보세요.\n  SAP Business Unleashed\n가상 이벤트 | 2025년 2월 14일\n자세히 알아보기\n귀사를 위해 구축된 솔루션\n제품 보기 A-Z\n전사적 자원 관리(ERP)\nSAP S/4HANA Cloud는 수상 실적에 빛나는 완벽한 모듈식 ERP입니다. 내장된 AI와 분석으로 어디서나 실시간으로 비즈니스 운영을 지원할 수 있습니다.\nSAP S/4HANA Cloud 시작하기 \nCRM 및 고객경험(CX)\nSAP의 CX 솔루션으로 전자상거래, 

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [12]:
# retriever = vector_db.as_retriever()

In [ ]:
# results = retriever.invoke("기업의 비전에 대해 알려주세요.",
#                            search_type="mmr",
#                            search_kwargs={"k" : 5, "fetch_k" : 10, "lambda_mult" : 0.5})

# for idx, result in enumerate(results):
#     print("=" * 100)
#     print(f"Result {idx:>04}")
#     print(f"{result.metadata['url']}\n{result.page_content}\n\n")

In [ ]:
# results = retriever.invoke("기업의 비전에 대해 알려주세요.",
#                            search_type="similarity_score_threshold",
#                            search_kwargs={"score_threshold" : 0.55})

# for idx, result in enumerate(results):
#     print("=" * 100)
#     print(f"Result {idx:>04}")
#     print(f"{result.metadata['url']}\n{result.page_content}\n\n")

In [ ]:
# results = vector_db.similarity_search_with_relevance_scores("현대자동차그룹의 비전에 대해 알려주세요.", k=10)

# for document, score in results:
#     print("=" * 100)
#     print(f"{document.id}, {document.metadata['url']}")
#     print(f"{score:.4f}")
#     print(f"{document.page_content}\n\n")

In [ ]:
# results = vector_db.similarity_search_with_relevance_scores("현대자동차그룹의 경영철학에 대해 알려주세요.", k=10)

# for document, score in results:
#     print("=" * 100)
#     print(f"{document.id}, {document.metadata['url']}")
#     print(f"{score:.4f}")
#     print(f"{document.page_content}\n\n")

In [ ]:
results = vector_db.similarity_search_with_score("현대자동차그룹의 비전에 대해 알려주세요.", k=10)

for document, score in results:
    print("=" * 100)
    print(f"{document.id}, {document.metadata['url']}")
    print(f"Score : {score:.4f}")
    print(f"{document.page_content}\n\n")

In [18]:
results = vector_store.similarity_search_with_score("현대자동차그룹의 비전에 대해 알려주세요.", k=10)

for document, score in results:
    print("=" * 100)
    print(f"{document.id}, {document.metadata['url']}")
    print(f"Score : {score:.4f}")
    print(f"{document.page_content}\n\n")

None, https://audit.hyundai.com/progress
Score : 0.9417
기업이념
윤리규범
제보하기
제보하기
추진경과
현대자동차그룹은 국민행복과 국가발전을 위해
‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
추진경과
추진경과
현대자동차그룹은 사회적 책임을 다하기 위해 윤리경영을 적극 실천하고 있습니다.
2023
2022
2021
2020
2019
2018
기아, 온실가스 감축 위한 ‘다자 협력’ 나선다
현대자동차, 서울시에 시각장애인 맞춤 복지차량 기증
현대모비스, 10년간 조성한 생태숲에서 멸종위기종 복원
현대자동차그룹, 임직원과 함께하는 기부 캠페인 ‘기부해봄’ 실시
현대글로비스, 지구의 날 맞아 서울숲 정화활동 플로깅 실시
현대건설, ‘CDP 코리아 어워드’ 기후변화 부문 2년 연속 최상위 등급 선정
사이트 맵
개인정보처리방침
한국 웹접근성 인증
서울특별시 서초구 헌릉로 12(양재동)
02-3464-3500
02-3464-8813
KOR
COPYRIGHT ⓒ HYUNDAI MOTOR GROUP. ALL RIGHTS RESERVED.


None, https://audit.hyundai.com/philosophy
Score : 0.9396
기업이념
윤리규범
제보하기
제보하기
기업이념체계
현대자동차그룹은 국민행복과 국가발전을 위해
‘모범적인 기업’으로서의 역할에 더욱 책임을 다하겠습니다.
기업이념
기업이념체계
현대자동차그룹은 창립 이래 우리만의 고유한 정신과 가치를 계승하고 실천함으로써 지속 성장을 이루어 왔습니다. 이렇듯 현대자동차그룹이 추구하고 지켜나가야 할 가치와 변화의 방향, 그리고 미래상을 유기적으로 구성한 ‘경영이념체계’는 또 하나의 역사를 만들어 가는 구성원 모두의 목표이자 구심점입니다. 경영철학, 비전, 핵심가치를 이해관계자들과 공유함으로써, 구체적인 실천을 약속합니다.
경영철학
핵심가치
비전
경영철학
경영철학 핵심개념
창의적 사고와 끝없는 도전을 통해 새로운 미래를 창조함으로써 인류사회의 꿈을 

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'es01'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
